### План реализации проекта

#### 1. Загрузка и первичный анализ данных

- Загрузим предоставленные CSV-файлы
- Проверим типы данных, наличие дубликатов id и пропусков значений
- Объединим информацию о коровах и быках, чтобы упростить работу с родословной

#### 2. Вменение пропущенных значений EBV

_Ключевой проблемой в данных является большое количество пропущенных значений EBV у коров и одного среди быков. Простое удаление этих данных или заполнение средним значением приведёт к потере ценной генетической информации. Попробуем восстановить (вменить) EBV на основе родословной. Если после нескольких итераций у животного невозможно посчитать EBV (например, у родителей тоже нет данных и так до основателей рода), в качестве крайней меры будем использовать средний EBV по всем животным (коровам\быкам), у которых он был изначально известен._

- Представим `pedigree.csv` в виде направленного графа, где узлы – это животные, а рёбра ведут от родителей к потомкам
- Для единственного быка с пропущенным EBV данных по родителям не оказалось, будем использовать среднее EBV по всем остальным быкам _(забегая вперёд, этот неизвестный бык в итоговую селективную подборку всё равно не вошёл)_
- Для каждой потенциальной пары "корова-бык" рассчитаем коэффициент инбридинга их гипотетического потомка

__Ограничения__
- Каждая корова закрепляется ровно за одним быком
- Ограничение на использование каждого быка _(10% от общего числа коров)_
- Ограничение по инбридингу _(используем бинарный параметр допустимости пары (1, если инбридинг ≤0.05, иначе 0))_

#### 3. Реализация жадного алгоритма

- Инициализация
   - Создадим список всех коров для осеменения
   - Создадим словарь, в котором будем фиксировать использование каждого быка
   - Установим максимальное значение использования быка
- Расчёт оценки для пар
   - Найдём средний EBV по всем возможным потомкам от всех _допустимых_ пар
   - Для каждой такой пары рассчитаем оценку, которая будет поощрять разнообразие и вознаграждать как высокий EBV, так и отклонение от среднего
- Итеративное применение
   - Запуск цикла, который будет продолжаться, пока не будет найдена идеальная пара для каждой коровы
   - Чтобы избежать случая, когда первые коровы в списке получат лучших быков, можно обрабатывать коров в случайном порядке, либо в порядке убывания их собственного EBV

#### 4. Формирование и проверка результата

- Создадим и сохраним выходной файл
- Ещё раз убедимся, что задействованы все коровы, а быки используются в рамках заданного ограничения
- Расчитаем некоторые статистики по полученным парам

### Python-скрипт

In [7]:
# Импорт необходимых библиотек
import numpy as np
import pandas as pd
import networkx as nx
from tqdm import tqdm
from functools import lru_cache

In [2]:
# Константы и параметры
DATA_PATH = './' # Путь к папке с данными

# Файлы с входными данными
BULLS_FILE = DATA_PATH + 'bulls.csv'
COWS_FILE = DATA_PATH + 'cows.csv'
PEDIGREE_FILE = DATA_PATH + 'pedigree.csv'

# Выходной файл
OUTPUT_FILE = 'cow_bull_assignments.csv'

# Параметры модели
INBREEDING_THRESHOLD = 0.05
BULL_USAGE_RATIO = 0.10
LAMBDA_SCORE = 0.7  # 70% важности для среднего EBV, 30% для разнообразия

In [ ]:
# Функции подготовки данных
def load_and_prepare_data(bulls_file, cows_file, pedigree_file):
    """
    Загружает и подготавливает исходные данные
    """
    print("Этап 1: Загрузка и подготовка данных...")
    bulls = pd.read_csv(bulls_file)
    cows = pd.read_csv(cows_file)
    pedigree_df = pd.read_csv(pedigree_file)

    pedigree_graph = nx.DiGraph()
    for _, row in pedigree_df.iterrows():
        animal_id, mother_id, father_id = row['id'], row['mother_id'], row['father_id']
        if pd.notna(mother_id):
            pedigree_graph.add_edge(mother_id, animal_id)
        if pd.notna(father_id):
            pedigree_graph.add_edge(father_id, animal_id)
            
    print(f"Загружено {len(bulls)} быков, {len(cows)} коров и {len(pedigree_df)} записей о родословной")
    print("Данные загружены, граф родословной построен\n")
    return bulls, cows, pedigree_df, pedigree_graph

def impute_ebv_values(bulls, cows, pedigree_df):
    """
    Вменяет пропущенные значения EBV, используя родословную
    """
    print("Этап 2: Вменение пропущенных значений EBV...")
    # Объединяем всех животных для удобства
    animals = pd.concat([
        bulls[['id', 'ebv']],
        cows[['id', 'ebv']]
    ]).set_index('id')

    # Обработка быка с пропущенным EBV
    bull_missing_ebv = bulls[bulls['ebv'].isna()]
    if not bull_missing_ebv.empty:
        bull_id = bull_missing_ebv['id'].iloc[0]
        mean_bull_ebv = bulls['ebv'].mean()
        animals.loc[bull_id, 'ebv'] = mean_bull_ebv
        print(f"EBV для быка {bull_id} был вменен средним значением по быкам: {mean_bull_ebv:.2f}")

    # Итеративное вменение EBV для коров на основе родителей
    ebv_dict = animals['ebv'].to_dict()
    pedigree_map = pedigree_df.set_index('id').T.to_dict()

    imputed_in_iteration = -1
    iteration = 0
    while imputed_in_iteration != 0:
        imputed_in_iteration = 0
        iteration += 1
        ids_to_impute = [id for id, val in ebv_dict.items() if pd.isna(val)]
        
        for animal_id in ids_to_impute:
            parents = pedigree_map.get(animal_id)
            if not parents:
                continue

            mother_id, father_id = parents.get('mother_id'), parents.get('father_id')
            
            mother_ebv = ebv_dict.get(mother_id)
            father_ebv = ebv_dict.get(father_id)

            if pd.notna(mother_ebv) and pd.notna(father_ebv):
                imputed_val = 0.5 * mother_ebv + 0.5 * father_ebv
                ebv_dict[animal_id] = imputed_val
                imputed_in_iteration += 1
        
        print(f"Итерация {iteration}: вменено {imputed_in_iteration} значений EBV")

    # Обработка оставшихся пропусков (основатели рода без EBV)
    remaining_missing_ids = [id for id, val in ebv_dict.items() if pd.isna(val)]
    if remaining_missing_ids:
        print(f"Осталось {len(remaining_missing_ids)} животных без EBV (вероятно, основатели рода)")
        # Используем среднее по коровам с известным изначальным EBV
        mean_cow_ebv = cows['ebv'].dropna().mean()
        for animal_id in remaining_missing_ids:
            ebv_dict[animal_id] = mean_cow_ebv
        print(f"Заполняем оставшиеся пропуски средним EBV коров: {mean_cow_ebv:.2f}")
    
    print("Вменение EBV завершено\n")
    return ebv_dict

In [ ]:
# Основной альгоритм подбора пар
def run_mating_algorithm(bulls, cows, ebv_dict, pedigree_graph):
    """
    Основной алгоритм подбора пар
    """
    print("Этап 3: Запуск алгоритма подбора пар...")
    
    # Максимальное количество осеменений на одного быка
    max_bull_usage = int(len(cows) * BULL_USAGE_RATIO)
    print(f"Ограничение на одного быка: не более {max_bull_usage} коров")

    # Создаем кешированную функцию для расчета инбридинга, чтобы избежать повторных вычислений
    @lru_cache(maxsize=None)
    def get_inbreeding_coefficient(animal1_id, animal2_id):
        if not pedigree_graph.has_node(animal1_id) or not pedigree_graph.has_node(animal2_id):
            return 0.0
        
        ancestors1 = nx.ancestors(pedigree_graph, animal1_id)
        ancestors2 = nx.ancestors(pedigree_graph, animal2_id)
        common_ancestors = ancestors1.intersection(ancestors2)

        if not common_ancestors:
            return 0.0

        inbreeding_sum = 0.0
        for ancestor in common_ancestors:
            # Находим длину пути (количество поколений) от каждого родителя до общего предка
            # Используем BFS для поиска кратчайшего пути вверх по графу
            try:
                path_len1 = len(nx.shortest_path(pedigree_graph, ancestor, animal1_id))
                path_len2 = len(nx.shortest_path(pedigree_graph, ancestor, animal2_id))
                n1, n2 = path_len1 - 1, path_len2 - 1
                inbreeding_sum += (0.5) ** (n1 + n2 + 1)
            except nx.NetworkXNoPath:
                # Такое может быть в неполных родословных, пропускаем
                continue
                
        return inbreeding_sum

    # Создание и фильтрация всех возможных пар
    print("Создание и фильтрация всех возможных пар...")
    valid_pairs = []
    bull_ids = bulls['id'].tolist()
    cow_ids = cows['id'].tolist()

    for cow_id in tqdm(cow_ids, desc="Проверка пар"):
        for bull_id in bull_ids:
            inbreeding_f = get_inbreeding_coefficient(cow_id, bull_id)
            if inbreeding_f <= INBREEDING_THRESHOLD:
                offspring_ebv = 0.5 * (ebv_dict[cow_id] + ebv_dict[bull_id])
                valid_pairs.append({
                    'cow_id': cow_id,
                    'bull_id': bull_id,
                    'offspring_ebv': offspring_ebv
                })

    print(f"Найдено {len(valid_pairs)} допустимых пар (инбридинг <= {INBREEDING_THRESHOLD * 100}%)")
    
    if not valid_pairs:
        raise ValueError("Не найдено ни одной допустимой пары. Проверьте порог инбридинга")

    # Расчёт оценки привлекательности для каждой пары
    pairs_df = pd.DataFrame(valid_pairs)
    global_mean_ebv = pairs_df['offspring_ebv'].mean()
    
    pairs_df['score'] = (LAMBDA_SCORE * pairs_df['offspring_ebv'] +
                         (1 - LAMBDA_SCORE) * abs(pairs_df['offspring_ebv'] - global_mean_ebv))
    
    # Жадный алгоритм назначения
    print("Запуск жадного алгоритма назначения...")
    pairs_df = pairs_df.sort_values(by='score', ascending=False)
    
    assignments = {}
    bull_usage_counts = {bull_id: 0 for bull_id in bull_ids}
    assigned_cows = set()

    for _, pair in tqdm(pairs_df.iterrows(), total=len(pairs_df), desc="Назначение пар"):
        cow_id, bull_id = pair['cow_id'], pair['bull_id']
        
        if cow_id not in assigned_cows and bull_usage_counts[bull_id] < max_bull_usage:
            assignments[cow_id] = bull_id
            bull_usage_counts[bull_id] += 1
            assigned_cows.add(cow_id)
            
        if len(assigned_cows) == len(cows):
            break # Все коровы назначены
            
    print("Алгоритм подбора пар завершён\n")
    return assignments, bull_usage_counts

In [5]:
# Главный блок выполнения
def main():
    """
    Главная функция для выполнения всего процесса
    """
    bulls, cows, pedigree_df, pedigree_graph = load_and_prepare_data(BULLS_FILE, COWS_FILE, PEDIGREE_FILE)
    ebv_dict = impute_ebv_values(bulls, cows, pedigree_df)
    
    bulls['ebv'] = bulls['id'].map(ebv_dict)
    cows['ebv'] = cows['id'].map(ebv_dict)

    assignments, bull_usage_counts = run_mating_algorithm(bulls, cows, ebv_dict, pedigree_graph)

    print("Этап 4: Формирование и проверка результата...")
    if len(assignments) != len(cows):
        print(f"ВНИМАНИЕ: Назначено {len(assignments)} из {len(cows)} коров. Возможно, стоит ослабить ограничения")

    assignments_df = pd.DataFrame(list(assignments.items()), columns=['cow_id', 'bull_id'])
    assignments_df.to_csv(OUTPUT_FILE, index=False)
    print(f"Результаты сохранены в файл: {OUTPUT_FILE}")

    # Финальная статистика
    assigned_bulls_ebv = assignments_df['bull_id'].map(ebv_dict)
    assigned_cows_ebv = assignments_df['cow_id'].map(ebv_dict)
    final_offspring_ebv = (assigned_bulls_ebv + assigned_cows_ebv) / 2
    
    print("\nИтоговая статистика по потомству")
    print(f"Средний ожидаемый EBV потомства: {final_offspring_ebv.mean():.2f}")
    print(f"Стандартное отклонение EBV потомства: {final_offspring_ebv.std():.2f}")
    print(f"Минимальный EBV потомства: {final_offspring_ebv.min():.2f}")
    print(f"Максимальный EBV потомства: {final_offspring_ebv.max():.2f}")
    
    print("\nСтатистика использования быков")
    usage_df = pd.Series(bull_usage_counts).sort_values(ascending=False)
    print(usage_df[usage_df > 0])
    
if __name__ == '__main__':
    main()

Этап 1: Загрузка и подготовка данных...
Загружено 39 быков, 17177 коров и 94400 записей о родословной
Данные загружены, граф родословной построен

Этап 2: Вменение пропущенных значений EBV...
EBV для быка GB00000052082 был вменен средним значением по быкам: 480.37
Итерация 1: вменено 95 значений EBV
Итерация 2: вменено 0 значений EBV
Осталось 1754 животных без EBV (вероятно, основатели рода)
Заполняем оставшиеся пропуски средним EBV коров: 362.15
Вменение EBV завершено

Этап 3: Запуск алгоритма подбора пар...
Ограничение на одного быка: не более 1717 коров
Создание и фильтрация всех возможных пар...


Проверка пар: 100%|████████████████████████████████████████████████████████████| 17177/17177 [1:39:36<00:00,  2.87it/s]


Найдено 643963 допустимых пар (инбридинг <= 5.0%)
Запуск жадного алгоритма назначения...


Назначение пар:  93%|███████████████████████████████████████████████████▎   | 600880/643963 [00:31<00:02, 19371.34it/s]


Алгоритм подбора пар завершен

Этап 4: Формирование и проверка результата...
Результаты сохранены в файл: cow_bull_assignments.csv

Итоговая статистика по потомству
Средний ожидаемый EBV потомства: 908.57
Стандартное отклонение EBV потомства: 357.70
Минимальный EBV потомства: -351.70
Максимальный EBV потомства: 1957.05

Статистика использования быков
US00000003459    1717
US00000003013    1717
US00000001653    1717
DE00000001742    1717
GB00000002348    1717
US00000002804    1717
FR00000002912    1717
US00000000795    1717
US00000003507    1717
GB00000002585    1669
DE00000003760      55
dtype: int64
